In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    
def trend(time, slope=0):
    return time*slope

def seasonal_pattern(season_time):
    return np.where(season_time<0.4,
                   np.cos(season_time*2*np.pi),
                   1/np.exp(3*season_time))

def seasonality(time, period, amplitude=1, phase=0):
    season_time=((time+phase)%period)/period
    return amplitude*seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd=np.random.RandomState(seed)
    return rnd.randn(len(time))*noise_level

time=np.arange(4*365+1, dtype="float32")
baseline=10
series=trend(time, 0.1)
baseline=10
amplitude=40
slope=0.05
noise_level=5

series=baseline+trend(time, slope)+seasonality(time, period=365, amplitude=amplitude)
series+=noise(time, noise_level, seed=42)

split_time=1000
time_train=time[:split_time]
x_train=series[:split_time]
time_valid=time[split_time:]
x_valid=series[split_time:]

window_size=20
batch_size=32
shuffle_buffer_size=1000

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset=tf.data.Dataset.from_tensor_slices(series)
    dataset=dataset.window(window_size+1, shift=1, drop_remainder=True)
    dataset=dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [10]:
dataset=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(dataset)
l0=tf.keras.layers.Dense(1, input_shape=[window_size])
model=tf.keras.models.Sequential([l0])

model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset, epochs=100, verbose=1)

print("Layer weights {}".format(l0.get_weights()))

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.float32, tf.float32)>
Epoch 1/100
31/31 [==============================] - 0s 2ms/step - loss: 118.6159
Epoch 2/100
31/31 [==============================] - 0s 1ms/step - loss: 102.3784
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 97.8642
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 93.7581
Epoch 5/100
31/31 [==============================] - 0s 1ms/step - loss: 90.6983
Epoch 6/100
31/31 [==============================] - 0s 1ms/step - loss: 88.0253
Epoch 7/100
31/31 [==============================] - 0s 1ms/step - loss: 84.5431
Epoch 8/100
31/31 [==============================] - 0s 2ms/step - loss: 82.4832
Epoch 9/100
31/31 [==============================] - 0s 1ms/step - loss: 80.9726
Epoch 10/100
31/31 [==============================] - 0s 6ms/step - loss: 78.2181
Epoch 11/100
31/31 [==============================] - 0s 7ms/step - loss: 77.0481
Epoch 12/100
31/31 [==

Epoch 100/100
31/31 [==============================] - 0s 2ms/step - loss: 44.0362
Layer weights [array([[-0.02427935],
       [-0.03594102],
       [ 0.06066339],
       [-0.08263371],
       [ 0.08729955],
       [ 0.06768211],
       [-0.03510593],
       [-0.06277171],
       [-0.02809489],
       [ 0.04383881],
       [ 0.04286271],
       [-0.04988132],
       [-0.0665676 ],
       [ 0.10072399],
       [ 0.07964637],
       [-0.04698017],
       [ 0.05048218],
       [ 0.12201331],
       [ 0.35027096],
       [ 0.41643512]], dtype=float32), array([0.01518231], dtype=float32)]


In [9]:
forecast = []

for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

KeyboardInterrupt: 

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()